In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
data=pd.read_csv('ANNs_file_Encrypt_Each_Line_Random_Key.csv')
data.head()

,Plain_text,Key,Cipher_text
0,like 2 weeks ago and it's going as terribly as...,b'\xda\x7f\xe9\xdf\x1fvz\xfb\xcd\xf2ty\x9b\xa7...,625d195f17e84a160e51e4c96b4796f63456f14e46b976...
1,"literally never about that account, love it.",b'q\x98\xc7\xeaw\x1f>\x90\xc5\xbajZ]\xb9l\xd6',d9e3479b9ded8368d1779d06d9b9e0a503e1d9a21ae409...
2,answer me this fellow apple peoples: how many ...,b'b\xbdO~\xb9Z.RE\xfa\xf3\xb7q\xbc\x91R',2e4deec4907057d1cd705fb8b1fb8b37325e551fca1bb2...
3,about 50 times today. terminal vim user.,b'-\xec\x07t\x1c\xff\x92L\x19\xefNO\x97\xc9-\x14',4f87a78c20c1e444b4d9063a4d29516ce16bfed6aaaea8...
4,seems the major complaints so far are from vim...,b'!\xcf\x0b\xfb2\rv\xc4\xf1m\x18\xfa\xcaC\x96\...,ee308cb97bf28a076735149bde3342639db98ca1d42177...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10513 entries, 0 to 10512
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Plain_text   10513 non-null  object
 1   Key          10513 non-null  object
 2   Cipher_text  10513 non-null  object
dtypes: object(3)
memory usage: 246.5+ KB


In [4]:
data.isnull().sum()

,0
Plain_text,0
Key,0
Cipher_text,0


In [5]:
def text_to_vector(x):
  return np.array([ord(i)/128 for i in x])

In [6]:
max_lenghth=max(data['Plain_text'].apply(len).max(),data['Cipher_text'].apply(len).max())
max_lenghth

288

In [7]:
data['Plain_text']=data['Plain_text'].apply(lambda x: np.pad(text_to_vector(x),(0,max_lenghth-len(x)),'constant'))
data['Cipher_text']=data['Cipher_text'].apply(lambda x: np.pad(text_to_vector(x),(0,max_lenghth-len(x)),'constant'))

In [8]:
data.head()

,Plain_text,Key,Cipher_text
0,"[0.84375, 0.8203125, 0.8359375, 0.7890625, 0.2...",b'\xda\x7f\xe9\xdf\x1fvz\xfb\xcd\xf2ty\x9b\xa7...,"[0.421875, 0.390625, 0.4140625, 0.78125, 0.382..."
1,"[0.84375, 0.8203125, 0.90625, 0.7890625, 0.890...",b'q\x98\xc7\xeaw\x1f>\x90\xc5\xbajZ]\xb9l\xd6',"[0.78125, 0.4453125, 0.7890625, 0.3984375, 0.4..."
2,"[0.7578125, 0.859375, 0.8984375, 0.9296875, 0....",b'b\xbdO~\xb9Z.RE\xfa\xf3\xb7q\xbc\x91R',"[0.390625, 0.7890625, 0.40625, 0.78125, 0.7890..."
3,"[0.7578125, 0.765625, 0.8671875, 0.9140625, 0....",b'-\xec\x07t\x1c\xff\x92L\x19\xefNO\x97\xc9-\x14',"[0.40625, 0.796875, 0.4375, 0.4296875, 0.75781..."
4,"[0.8984375, 0.7890625, 0.7890625, 0.8515625, 0...",b'!\xcf\x0b\xfb2\rv\xc4\xf1m\x18\xfa\xcaC\x96\...,"[0.7890625, 0.7890625, 0.3984375, 0.375, 0.437..."


In [9]:
y=np.stack(data['Plain_text'].values)
x=np.stack(data['Cipher_text'].values)

In [10]:
# ss=StandardScaler()
# x=ss.fit_transform(x)
# y=ss.fit_transform(y)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [12]:
model=Sequential()
model.add(Dense(256,activation='relu',input_dim=x_train.shape[1]))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(max_lenghth,activation='linear'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])

In [16]:
model.fit(x_train,y_train,epochs=100,batch_size=10,verbose=1)
loss,accuracy=model.evaluate(x_test,y_test)
accuracy,loss

Epoch 1/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1245 - loss: 0.1258
Epoch 2/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1257 - loss: 0.1270
Epoch 3/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1307 - loss: 0.1245
Epoch 4/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1233 - loss: 0.1262
Epoch 5/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1255 - loss: 0.1249
Epoch 6/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1245 - loss: 0.1250
Epoch 7/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1263 - loss: 0.1245
Epoch 8/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1245 - loss: 0.1259
Epoch 9/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1213 - loss: 0.1265
Epoch 10/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1280 - loss: 0.1272
Epoch 11/100
841/841 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1304 - loss: 0.1253
Epoch 12/100
841/841 ━━━━━━━━━━━━━━━━━━━━

(0.12696148455142975, 0.12845176458358765)